In [29]:
import pandas as pd
import json
import matplotlib.pyplot as plt 
import numpy as np

In [35]:
leagues = ["EPL", "La_liga", "Bundesliga", "Serie_A", "Ligue_1"]

team_cols = [
    "league",
    "team_id",
    "team_title",
    "gameweek",
    "h_a",
    "xG",
    "xGA",
    "npxG",
    "npxGA",
    "ppda",
    "ppda_allowed",
    "deep",
    "deep_allowed",
    "scored",
    "conceded",
    "xpts",
    "result",
    "date",
    "wins",
    "draws",
    "loses",
    "pts",
    "npxGD",
    "adv_id",
    "adv_title",
    "forecast_w",
    "forecast_d",
    "forecast_l"
]
team_data_flatten = []

for league in leagues:
    dates_data = json.load(open(f"data/{league}_datesData.json"))

    team_data = json.load(open(f"data/{league}_teamsData.json"))
    team_data = list(team_data.values())


    
    for team in team_data:
        for i, history in enumerate(team["history"]):
            team_d = [league, team["id"], team["title"], i + 1]

            h_a = history["h_a"]

            team_d.append(history["h_a"])
            team_d.append(history["xG"])
            team_d.append(history["xGA"])
            team_d.append(history["npxG"])
            team_d.append(history["npxGA"])
            team_d.append(np.round(history["ppda"]["att"]/history["ppda"]["def"], decimals=2))
            team_d.append(np.round(history["ppda_allowed"]["att"]/history["ppda_allowed"]["def"], decimals=2))
            team_d.append(history["deep"])
            team_d.append(history["deep_allowed"])
            team_d.append(history["scored"])
            team_d.append(history["missed"])
            team_d.append(history["xpts"])
            team_d.append(history["result"])
            team_d.append(history["date"])
            team_d.append(history["wins"])
            team_d.append(history["draws"])
            team_d.append(history["loses"])
            team_d.append(history["pts"])
            team_d.append(history["npxGD"])

            dates_match = [d for d in dates_data if d["datetime"] == history["date"] and d[h_a]['id'] == team["id"]]
            
            if len(dates_match) == 1:
                match = dates_match[0]
                adv = match["h"] if h_a == "a" else match["a"]
                team_d.append(adv["id"])
                team_d.append(adv["title"])
                team_d.append(float(match["forecast"]["w"] if h_a == "a" else match["forecast"]["l"]))
                team_d.append(float(match["forecast"]["d"]))
                team_d.append(float(match["forecast"]["w"] if h_a == "h" else match["forecast"]["l"]))

            team_data_flatten.append(team_d)

df_teams = pd.DataFrame(team_data_flatten, columns=team_cols)

    # df_teams[((df_teams["team_id"] == "94") & (df_teams["gameweek"] == 1)) | ((df_teams["team_id"] == "105") & (df_teams["gameweek"] == 1))].head(10)

In [36]:
df_teams['xG_diff'] = df_teams['xG'] - df_teams['scored']
df_teams['xGA_diff'] = df_teams['xGA'] - df_teams['conceded']

In [37]:
df_teams.head()

,league,team_id,team_title,gameweek,h_a,xG,xGA,npxG,npxGA,ppda,...,loses,pts,npxGD,adv_id,adv_title,forecast_w,forecast_d,forecast_l,xG_diff,xGA_diff
0,EPL,71,Aston Villa,1,a,2.67030,2.804700,2.67030,2.043530,11.30,...,0,3,0.626770,81,West Ham,0.4123,0.2321,0.3556,0.67030,1.804700
1,EPL,71,Aston Villa,2,h,1.31664,1.413990,1.31664,1.413990,17.28,...,1,0,-0.097350,83,Arsenal,0.3864,0.3023,0.3113,1.31664,-0.586010
2,EPL,71,Aston Villa,3,a,2.16385,0.774741,2.16385,0.774741,10.47,...,0,3,1.389109,75,Leicester,0.0646,0.1791,0.7563,0.16385,-0.225259
3,EPL,71,Aston Villa,4,h,3.15876,0.852158,3.15876,0.852158,5.26,...,0,3,2.306602,72,Everton,0.0243,0.0838,0.8919,0.15876,-1.147842
4,EPL,71,Aston Villa,5,h,2.37351,0.520117,2.37351,0.520117,11.80,...,0,3,1.853393,229,Wolverhampton Wanderers,0.0417,0.1134,0.8449,-0.62649,-0.479883


In [38]:
df_teams[df_teams["league"] == "Serie_A"].head()

,league,team_id,team_title,gameweek,h_a,xG,xGA,npxG,npxGA,ppda,...,loses,pts,npxGD,adv_id,adv_title,forecast_w,forecast_d,forecast_l,xG_diff,xGA_diff
370,Serie_A,94,Verona,1,h,1.805360,0.975176,1.805360,0.975176,15.33,...,0,3,0.830184,105,Napoli,0.1481,0.2346,0.6173,-1.194640,0.975176
371,Serie_A,94,Verona,2,h,0.878611,1.361510,0.878611,0.600212,19.38,...,1,0,0.278399,98,Juventus,0.5017,0.3109,0.1874,0.878611,-1.638490
372,Serie_A,94,Verona,3,a,2.096530,1.347350,1.335240,1.347350,13.62,...,0,3,-0.012110,101,Genoa,0.1811,0.2480,0.5709,0.096530,1.347350
373,Serie_A,94,Verona,4,a,0.533276,1.792430,0.533276,1.792430,18.76,...,1,0,-1.259154,96,Lazio,0.7489,0.1917,0.0594,-0.466724,-0.207570
374,Serie_A,94,Verona,5,h,0.777792,1.966820,0.777792,1.205520,33.46,...,1,0,-0.427728,113,Torino,0.7068,0.2013,0.0919,-1.222208,-1.033180


In [39]:
df_teams.to_csv("data/full_data.csv", index=False)